In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [6]:
!pip install konlpy
!pip install tomotopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.1-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (465 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import re
import tomotopy as tp
import os
from konlpy.tag import Okt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import MinMaxScaler
import progressbar


plt.rc('font', family='NanumBarunGothic') 


In [8]:
main_path = '/content/drive/MyDrive/Project/LiveCommerse/'
# main_path = '/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/'

In [10]:
df = pd.read_excel(main_path + 'FUCKUGOOGLE/Tour_mall_total_2.xlsx')

FileNotFoundError: ignored

In [ ]:
df

In [ ]:
class ExpNormalize(clr.Normalize):
    def __init__(self, scale):
        super().__init__()
        self.scale = scale

    def __call__(self, value, clip=None):
        if clip is None:
            clip = self.clip

        result, is_scalar = self.process_value(value)

        self.autoscale_None(result)
        (vmin,), _ = self.process_value(self.vmin)
        (vmax,), _ = self.process_value(self.vmax)
        if vmin == vmax:
            result.fill(0)
        elif vmin > vmax:
            raise ValueError("minvalue must be less than or equal to maxvalue")
        else:
            if clip:
                mask = np.ma.getmask(result)
                result = np.ma.array(np.clip(result.filled(vmax), vmin, vmax),
                                    mask=mask)
            resdat = result.data
            resdat = 1 - np.exp(-2 * resdat / self.scale)
            result = np.ma.array(resdat, mask=result.mask, copy=False)
        if is_scalar:
            result = result[0]
        return result

heat = clr.LinearSegmentedColormap.from_list('heat', 
    [(0, 0, 0), (0, 0, 1), (0, 1, 1), (0, 1, 0), (1, 1, 0), (1, 0, 0), (1, 1, 1)],
    N=1024
)

In [ ]:
maxsub = df.Channel_Sub.max()
minsub = df.Channel_Sub.min()
maxvalue = df.Channel_Sentiment_Average.max()
minvalue = df.Channel_Sentiment_Average.min()
df['norm_senti'] = 0
df['norm_sub'] = 0

for i in range(len(df)):
  senti_avg = df.loc[i, 'Channel_Sentiment_Average']
  sub = df.loc[i, 'Channel_Sub']
  to_chat = df.loc[i, 'Channel_Total_Chat']
  chat_user = df.loc[i, 'Channel_Chat_User']
  df.loc[i, 'norm_sub'] = (sub - minsub) / (maxsub - minsub)
  df.loc[i, 'norm_senti'] = (senti_avg - minvalue) / (maxvalue - minvalue)

corpus = tp.utils.Corpus()
for i in range(len(df)):
  try:
    data = df.loc[i, 'words']
    senti_aver = df.loc[i,'norm_senti']
    metadt = df.loc[i, 'Chat_User_rank']
    sub = df.loc[i, 'norm_sub']
    corpus.add_doc(data.split(), numeric_metadata = [float(senti_aver). float(sub)])
    print(f'done : {i}')
  except:
    continue

mdl = tp.GDMRModel(tw=tp.TermWeight.ONE, k=30, degrees=[4, 3], 
    alpha=1e-2, sigma=0.25, min_cf = 0, min_df=1, rm_top = 12, sigma0=3.0, decay=1.0,
    eta = 1,
    metadata_range=[(0,1), (0,1)], corpus=corpus
)

In [ ]:
corpus[0]

In [ ]:
maxsub = df.Channel_Sub.max()
minsub = df.Channel_Sub.min()
maxvalue = df.Channel_Sentiment_Average.max()
minvalue = df.Channel_Sentiment_Average.min()
df['norm_senti'] = 0
df['norm_sub'] = 0

for i in range(len(df)):
  senti_avg = df.loc[i, 'Channel_Sentiment_Average']
  sub = df.loc[i, 'Channel_Sub']
  to_chat = df.loc[i, 'Channel_Total_Chat']
  chat_user = df.loc[i, 'Channel_Chat_User']
  df.loc[i, 'norm_sub'] = (sub - minsub) / (maxsub - minsub)
  df.loc[i, 'norm_senti'] = (senti_avg - minvalue) / (maxvalue - minvalue)

corpus = tp.utils.Corpus()
for i in range(len(df)):
  try:
    data = df.loc[i, 'words']
    senti_aver = df.loc[i,'norm_senti']
    metadt = df.loc[i, 'Chat_User_rank']
    sub = df.loc[i, 'norm_sub']
    corpus.add_doc(data.split(), numeric_metadata = [float(senti_aver). float(sub)])
    print(f'done : {i}')
  except:
    continue

mdl = tp.GDMRModel(tw=tp.TermWeight.ONE, k=30, degrees=[4, 3], 
    alpha=1e-2, sigma=0.25, min_cf = 0, min_df=1, rm_top = 12, sigma0=3.0, decay=1.0,
    eta = 1,
    metadata_range=[(0,1), (0,1)], corpus=corpus
)

# corpus = tp.utils.Corpus()
# for line in open('/content/drive/MyDrive/LiveCommerse/g-dmr-master/data/dataset2.txt', encoding='utf-8'):
#     fd = line.strip().split()
#     corpus.add_doc(fd[2:], numeric_metadata=list(map(float, fd[:2])))

# # We set a range of the first metadata as [2000, 2017] 
# # and one of the second metadata as [0, 1].
# mdl = tp.GDMRModel(tw=tp.TermWeight.PMI, k=30, degrees=[4, 3], 
#     alpha=1e-2, sigma=0.25, sigma0=3.0,
#     metadata_range=[(2000, 2017), (0, 1)], corpus=corpus
# )
mdl.optim_interval = 20
mdl.burn_in = 200

mdl.train(0)

print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(mdl.docs), len(mdl.used_vocabs), mdl.num_words
))

# Let's train the model
for i in range(0, 1000, 20):
    print('Iteration: {:04} LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)
print('Iteration: {:04} LL per word: {:.4}'.format(1000, mdl.ll_per_word))

mdl.summary()

# Let's visualize the result
topic_counts = mdl.get_count_by_topics()
lambdas = mdl.lambdas

md_range = mdl.metadata_range
# Our topic distribution map has 
# 400 pixels for the first axis and 
# 200 pixels for the second axis.
r = mdl.tdf_linspace(
    [md_range[0][0], md_range[1][0]], 
    [md_range[0][1], md_range[1][1]], 
    [400, 200]
)

for k in (-topic_counts).argsort():
    print('Topic #{} ({})'.format(k, topic_counts[k]))
    print(*(w for w, _ in mdl.get_topic_words(k)))
    print('Lambda:', lambdas[k])

    imgplot = plt.imshow(r[:, :, k].transpose(), clim=(0.0, r[:, :, k].max()), 
        origin='lower', cmap=heat, norm=ExpNormalize(scale=0.04),
        extent=[*md_range[0], *md_range[1]],
        aspect='auto'
    )
    plt.title('#{}\n({})'.format(k, ' '.join(w for w, _ in mdl.get_topic_words(k, top_n=5))))
    plt.colorbar()
    plt.show()